In [3]:
!pip install spacy
!pip install -U spacy
!pip install gradio
!python -m spacy download en_core_web_sm
!pip install wordcloud

Traceback (most recent call last):
  File "C:\Users\Abuoden\anaconda3\lib\runpy.py", line 188, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "C:\Users\Abuoden\anaconda3\lib\runpy.py", line 147, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "C:\Users\Abuoden\anaconda3\lib\runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "C:\Users\Abuoden\anaconda3\lib\site-packages\spacy\__init__.py", line 13, in <module>
    from . import pipeline  # noqa: F401
  File "C:\Users\Abuoden\anaconda3\lib\site-packages\spacy\pipeline\__init__.py", line 1, in <module>
    from .attributeruler import AttributeRuler
  File "C:\Users\Abuoden\anaconda3\lib\site-packages\spacy\pipeline\attributeruler.py", line 8, in <module>
    from ..language import Language
  File "C:\Users\Abuoden\anaconda3\lib\site-packages\spacy\language.py", line 43, in <module>
    from .pipe_analysis import analyze_pipes, p

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import string
import pickle
import os
import shutil
from wordcloud import WordCloud
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from scipy.stats import ttest_ind
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import TransformerMixin 
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

TypeError: issubclass() arg 1 must be a class

## Data Collection & Loading

In [ ]:
data=pd.read_csv('C:Assets\Train.csv')

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.info

In [ ]:
data.describe

In [ ]:
data.isna().sum()

In [ ]:
print(f'Columns Names: {list(data.columns)}')

In [ ]:
data1=pd.read_csv('C:Assets\Test.csv')

In [ ]:
data1

In [ ]:
data.info

In [ ]:
data.rename(columns={'content': 'review'}, inplace=True)

In [ ]:
data.drop('review_file', axis=1, inplace=True)

In [ ]:
data

## EDA

In [ ]:
# Distribution of sentiment labels
sns.countplot(x='sentiment', data=data)
plt.title('Distribution of Sentiment Labels')
plt.show()

In [ ]:
# Count the number of each sentiment label
sentiment_count = data['sentiment'].value_counts()

print(sentiment_count)

## HYPOTHESIS TESTING

### Question: Is there a significant difference in the average word count between positive and negative reviews

### H(0) - There is no significant difference in the average word count between positive and negative reviews 

### H(1)- There is a significant difference in the average word count between positive and negative reviews 

In [ ]:
# Separate positive and negative reviews
positive_reviews = data[data['sentiment'] == 'positive']
negative_reviews = data[data['sentiment'] == 'negative']

# Calculate word count for each review
positive_reviews['word_count'] = positive_reviews['review'].apply(lambda x: len(x.split()))
negative_reviews['word_count'] = negative_reviews['review'].apply(lambda x: len(x.split()))

# Perform a two-sample t-test
t_stat, p_value = ttest_ind(positive_reviews['word_count'], negative_reviews['word_count'])

# Define the significance level (alpha)
alpha = 0.05

# Check the p-value
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in word count between positive and negative reviews.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in word count between positive and negative reviews.")


  ## ANSWERING QUESTIONS

In [ ]:
# Plot the distribution of sentiment labels
sentiment_counts = data['sentiment'].value_counts()
plt.bar(sentiment_counts.index, sentiment_counts.values)
plt.title('Distribution of Sentiment Labels')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

## Feature Processing & Engineering

In [ ]:
# Load spaCy
# nlp = spacy.load("en_core_web_sm")
nlp = English()
stopwords = list(nlp.Defaults.stop_words)
punctuations = string.punctuation

In [ ]:
# Preprocess the text before splitting
data['review'] = data['review'].apply(lambda text: text.strip().lower())

In [ ]:
# Tokenization, stopwords removal, and punctuation removal using spaCy
#def process_text(text):
   # doc = nlp(text)
    #tokens = [token.text for token in doc if token.text not in stopwords and token.text not in punctuations]
    #return ' '.join(tokens)

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Additional stopwords and punctuations
additional_stopwords = {"film", "movie"}  # Customize this list based on your data
stopwords = list(STOP_WORDS.union(additional_stopwords))
punctuations = string.punctuation

# Function for processing text
def process_text(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.text.lower() not in stopwords and token.text not in punctuations:
            if token.lemma_ == '-PRON-':  # Handle pronouns
                tokens.append(token.text)
            else:
                tokens.append(token.lemma_)
    # Handling negation
    processed_tokens = []
    is_negation = False
    for token in tokens:
        if token in ["not", "no", "n't", "never"]:
            is_negation = not is_negation
        else:
            if is_negation and token in ["good", "great", "excellent"]:  # Adjust for positive words affected by negation
                token = "bad_" + token
            elif is_negation and token in ["bad", "poor", "worst"]:  # Adjust for negative words affected by negation
                token = "good_" + token
            processed_tokens.append(token)
    return ' '.join(processed_tokens)

# Apply the updated processing function to the 'review' column
data['processed_review'] = data['review'].apply(process_text)


## Data Splitting

In [ ]:
# Split the data into training and testing sets
X = data['review']
y = data['sentiment']

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Feature Encoding

In [ ]:
#  #Tokenization, stopwords removal, and punctuation removal using spaCy
# def process_text(text):
#     doc = nlp(text)
#     tokens = [token.text for token in doc if token.text not in stopwords and token.text not in punctuations]
#     return ' '.join(tokens)

In [ ]:
X_train = X_train.apply(process_text)
X_test = X_test.apply(process_text)

In [ ]:
# Text vectorization using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

## TRAIN & EVALUATE MODEL

In [ ]:
# # Train a sentiment classifier 
# classifier = MultinomialNB()
# classifier.fit(X_train_tfidf, y_train)

# # Make predictions on the test set
# y_pred = classifier.predict(X_test_tfidf)

# # Print the classification report
# class_report = classification_report(y_test, y_pred)
# print("Classification Report:")
# print(class_report)

# # Calculate and print the accuracy score
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy Score: {accuracy:.2f}")

In [ ]:
# List of classifiers
classifiers = {
    "MultinomialNB": MultinomialNB(),
    "LogisticRegression": LogisticRegression(),
    "RandomForest": RandomForestClassifier()
}

# Train and evaluate each classifier
for clf_name, classifier in classifiers.items():
    classifier.fit(X_train_tfidf, y_train)
    y_pred = classifier.predict(X_test_tfidf)

    # Print the classification report
    class_report = classification_report(y_test, y_pred)
    print(f"Classification Report for {clf_name}:")
    print(class_report)

    # Calculate and print the accuracy score in percentage format
    accuracy = accuracy_score(y_test, y_pred) * 100
    print(f"Accuracy Score for {clf_name}: {accuracy:.2f}%\n")

## CONFUSION MATRIX

In [ ]:
# # Train a sentiment classifier 
# classifier = MultinomialNB()
# classifier.fit(X_train_tfidf, y_train)

# # Make predictions on the test set
# y_pred = classifier.predict(X_test_tfidf)

# # Calculate the confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred)

# # Print the confusion matrix
# print("Confusion Matrix:")
# print(conf_matrix)

# # Visualize the confusion matrix 
# plt.figure(figsize=(8, 6))
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

In [ ]:
for clf_name, classifier in classifiers.items():
    classifier.fit(X_train_tfidf, y_train)
    y_pred = classifier.predict(X_test_tfidf)
    
    # Create the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Print the confusion matrix distribution
    print(f"Confusion Matrix for {clf_name} Distribution:")
    print(pd.DataFrame(cm, index=['Actual Negative', 'Actual Positive'], columns=['Predicted Negative', 'Predicted Positive']))

    # Plot the confusion matrix as a heatmap
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
    plt.title(f'Confusion Matrix for {clf_name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

## HYPERPARAMETER TUNING

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l1', 'l2']  # Regularization type
}

# Create the logistic regression model with random state
logistic_classifier = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(logistic_classifier, param_grid, cv=5, scoring='accuracy')

# Perform grid search
grid_search.fit(X_train_tfidf, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the final model with the best hyperparameters
best_classifier = LogisticRegression(solver='liblinear', max_iter=1000, **best_params)
best_classifier.fit(X_train_tfidf, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

# Make predictions on the test set using the tuned model
y_pred = best_classifier.predict(X_test_tfidf)

# Calculate and print the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy Score after Hyperparameter Tuning: {accuracy:.2f}")


In [ ]:
# Create a dictionary to store the components and model
components_dict = {
    'cleaner': process_text,
    'cleaner_tfidf_vectorizer': tfidf_vectorizer,
    'cleaner_classifier': best_classifier
}

In [ ]:
# Create a folder to store the exported data
folder_name = 'comp_folder'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
# Export the components and model
components_path = os.path.join(folder_name, 'sentiment_components.pkl')
with open(components_path, 'wb') as file:
    pickle.dump(components_dict, file)

In [ ]:
# Generate the requirements.txt file
requirements_file = 'requirements.txt'
os.system(f'pip freeze > {os.path.join(folder_name, requirements_file)}')

In [ ]:
# Zip the exported data folder
shutil.make_archive('exported_data', 'zip', 'exported_comp_folder')

print("Exported data has been zipped.")

### Test Data

In [ ]:
# Load the exported components from the file
components_path = 'comp_folder/sentiment_components.pkl'
with open(components_path, 'rb') as file:
    components_dict = pickle.load(file)

In [ ]:
# Extract the loaded components
loaded_cleaner = components_dict['cleaner']
loaded_vectorizer = components_dict['cleaner_tfidf_vectorizer']
loaded_classifier = components_dict['cleaner_classifier']

In [ ]:
# Load the new data from a CSV file
data1

In [ ]:
data1.rename(columns={'content': 'review'}, inplace=True)

In [ ]:
files = data1['review_file'].copy()

In [ ]:
data1.drop('review_file', axis=1, inplace=True)

In [ ]:
data1['review'] = data1['review'].apply(lambda text: text.strip().lower())

In [ ]:
# Apply text cleaning to the new data
data1['review'] = data1['review'].apply(loaded_cleaner)

# Vectorize the new data using the loaded vectorizer
X_new_tfidf = loaded_vectorizer.transform(data1['review'])

# Make predictions on the new data
new_predictions = loaded_classifier.predict(X_new_tfidf)

In [ ]:
# Add the "review_file" column back to the data1 DataFrame
data1['review_file'] = files

# Add the predictions to the new_data DataFrame
data1['predicted_sentiment'] = new_predictions

In [ ]:
# Create a new DataFrame with only "review_file" and "predicted_sentiment" columns
prediction = data1[['review_file', 'predicted_sentiment']]

In [ ]:
data1

In [ ]:
prediction

In [ ]:
# Count the occurrences of "Positive" and "Negative" in the predictions
count = prediction['predicted_sentiment'].value_counts()

print(count)

In [ ]:
# Save the Predictions
prediction.to_csv('predicted_sentiment_results.csv', index=False)

In [ ]:
# !pip show spacy

In [ ]:
import gradio as gr
import joblib
import pickle
# from text_preprocessing import process_text


# Load the exported components
components_path = r'src\assets\ML\sentiment_components.pkl'
with open(components_path, 'rb') as file:
    components_dict = pickle.load(file)


loaded_cleaner = components_dict['cleaner']
loaded_vectorizer = components_dict['cleaner_tfidf_vectorizer']
loaded_classifier = components_dict['cleaner_classifier']

def sent_analysis(text):
    # Preprocess the input text using the loaded cleaner
    text = loaded_cleaner(text)
    
    # Vectorize the text using the loaded vectorizer
    X_tfidf = loaded_vectorizer.transform([text])
    
    # Make predictions using the loaded classifier
    prediction = loaded_classifier.predict(X_tfidf)[0]
    
    # Return the predicted sentiment label
    return prediction

# Create a Gradio interface
demo = gr.Interface(
    fn=sent_analysis,
    inputs=gr.Textbox(placeholder="Enter a movie review..."),
    outputs="label",
    interpretation="default",
    examples=[
        ["I loved the movie! It was fantastic."],
        ["The acting was terrible, and the plot was boring."],
        ["This film is just okay. Not great, not terrible."],
    ],
    title="Movie Review Sentiment Analysis",
    description="An AI model that predicts sentiment in movie reviews, providing labels for 'NEGATIVE' and 'POSITIVE' sentiments.",
    theme="default",
    live=True,
)


if __name__ == "__main__":
    demo.launch()
    
!python src/app.py